In [4]:
%matplotlib inline

import pickle as pkl
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [6]:
#from tensorflow.examples.tutorials.mnist  import  input_data
#mnist = input_data.read_data_sets('../99.MNIST_data')

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [25]:
# real_dim : 本物データの次元
# z_dim : フェイクデータの次元
def model_imputs(real_dim, z_dim):
    inputs_real = tf.placeholder(tf.float32, (None, real_dim), name='input_real')
    inputs_z = tf.placeholder(tf.float32, (None, z_dim), name='input_z')
    
    return inputs_real, inputs_z

### generator関数

In [17]:
# reuse : False 呼び出すたびに、関数のなかの前回実行時の値を保持するかどうかを指定
def generator(z, out_dim, n_units=128, reuse=False, alpha=0.01):
    with tf.variable_scope('generator', reuse=reuse):
        h1 = tf.layers.dense(z, n_units, activation=None)
        h1 = tf.maximum(alpha * h1, h1) # Leaky ReLU
        
        logits = tf.layers.dense(h1, out_dim, activation=None)
        out = tf.tanh(logits)
        
        return out 

### discriminator関数の定義

In [44]:
# reuse : False 呼び出すたびに、関数のなかの前回実行時の値を保持するかどうかを指定
def discriminator(z, n_units=128, reuse=False, alpha=0.01):
    with tf.variable_scope('discriminator', reuse=reuse): # discriminatorと名前をつけて呼ばれた場合は前回の演算の値を保持する
        h1 = tf.layers.dense(z, n_units, activation=None)
        h1 = tf.maximum(alpha * h1, h1) # Leaky ReLU
        
        logits = tf.layers.dense(h1, 1, activation=None)
        out = tf.sigmoid(logits)
        
        return out, logits
    

In [45]:
# パイパーパラメータの初期化
input_size = 784
z_size = 100
g_hidden_size = 128
d_hidden_size = 128
alpha = 0.01
smooth = 0.1


In [43]:
# モデルの定義
tf.reset_default_graph()

input_real, input_z = model_imputs(input_size, z_size)

g_model = generator(input_z, input_size, n_units=g_hidden_size, alpha=alpha)

d_model_real, d_logits_real = discriminator(input_real, n_units=d_hidden_size, alpha=alpha)
d_model_fake, d_logits_fake = discriminator(g_model, reuse=True, n_units=d_hidden_size, alpha=alpha)


In [35]:
# 損失関数を定義（D/G）
# 正解から離れていると値が大きく発散する
d_loss_real = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real,
                                                                      labels=tf.ones_like(d_logits_real) *(1-smooth)))
d_loss_fake = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                                      labels=tf.zeros_like(d_logits_real)))

d_loss = d_loss_real  + d_loss_fake

# ランダムに生成された画像が本物として認識された誤差
g_loss =  tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                                labels=tf.ones_like(d_logits_fake)))


In [42]:
# 最適化手法を定義
learning_rate = 0.002

t_vars = tf.trainable_variables() # weight / bias を取り出す
g_vars = [var for var in t_vars if var.name.startswith('generator')]
d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

d_train_optimaize = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)
g_train_optimaize = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)
